# Live Demo with cam


In [4]:

from tensorflow import keras
print(keras.__version__)

model = keras.models.load_model('model/')

2.4.0


In [5]:
import pandas as pd

def labels():
    res = []
    for i in range(0, 28*28):
        res.append("pixel" + str(i))
    return res

In [6]:
import cv2
import numpy as np

cap = cv2.VideoCapture()
cap.open(0, cv2.CAP_DSHOW)

def get_img_contour_thresh(img):
    x, y, w, h = 15, 15, 300, 300
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    ret, thresh1 = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh1 = thresh1[y:y + h, x:x + w]
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    return img, contours, thresh1

try:
    while(True):
        status, frame = cap.read()
        if status:
            
            img, contours, thresh = get_img_contour_thresh(frame)
        
            if len(contours) > 0:
                contour = max(contours, key=cv2.contourArea)
                
                if cv2.contourArea(contour) > 2500:
                    
                    x, y, w, h = cv2.boundingRect(contour)

                    newImage = thresh[y:y + h, x:x + w]
                    
                    resized_image = cv2.resize(newImage, (28, 28))
                    df = pd.DataFrame([resized_image.flatten()],columns=labels())
                    to_predict = df.values
                    to_predict = to_predict/255
                    to_predict = to_predict.reshape((1,28, 28,1))

                    prediction = model.predict_classes(to_predict)[0]

            
            x, y, w, h = 15, 15, 300, 300
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 4)

            cv2.putText(img, " The number is : " + str(prediction), (10, 345),
                    cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 1)

            #change the window size to fit screen properly
            img = cv2.resize(img, (1000, 600))
            cv2.imshow("Frame", img)
            cv2.imshow("Contours", thresh)        

            
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            raise Exception('quiting window...')

except:
    cap.release()
    cv2.destroyAllWindows()
